

```
# This is formatted as code
```

# DMA Fall 20

**Note** : This entire lab will be manually evaluated.

Name : 'Victor Shi'
Collaborator : ''

# Lab 4: Neural Networks #


In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
!wget http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
!wget http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2020-09-23 18:58:53--  http://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
Resolving people.ischool.berkeley.edu (people.ischool.berkeley.edu)... 128.32.78.16
Connecting to people.ischool.berkeley.edu (people.ischool.berkeley.edu)|128.32.78.16|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv [following]
--2020-09-23 18:58:53--  https://people.ischool.berkeley.edu/~zp/course_datasets/lab_4_training.csv
Connecting to people.ischool.berkeley.edu (people.ischool.berkeley.edu)|128.32.78.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105581 (103K) [text/csv]
Saving to: ‘lab_4_training.csv’

lab_4_training.csv  100%[===================>] 103.11K   490KB/s    in 0.2s    

2020-09-23 18:58:54 (490 KB/s) - ‘lab_4_training.csv’ saved [105581/105581]

URL transformed to HTTPS due to an HSTS policy
--2020-09-23 18:58:54--  https://

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,1303,male,20,second,green,73.0,210.0,0,1,10.0,Yes,5.0,50.0,1.0,15.0
1,36,male,20,third,other,71.0,90.0,1,0,15.0,Yes,4.0,10.0,0.0,1.0
2,489,male,22,fourth,hazel,75.0,200.0,0,1,1.0,Yes,2.0,150.0,1.0,10.0
3,1415,male,19,second,brown,72.0,35.0,2,2,20.0,Yes,5.0,100.0,0.0,7.0
4,616,male,22,fourth,hazel,71.0,15.0,2,1,10.0,Yes,7.0,10.0,0.0,5.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of 'gender'. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives)

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [ ]:
# YOUR CODE HERE
majority = df_train['gender'].value_counts().idxmax()
train_target = df_train['gender'].value_counts()
tp = train_target[majority]
tn = 0
all = sum(train_target)
accuracy = (tp+tn)/all
print(accuracy)

0.5377358490566038


###ANSWER: 0.5377358490566038

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [ ]:
# YOUR CODE HERE

test_target = df_test['gender'].value_counts()
tp = test_target['female']
tn = 0
all = sum(test_target)
accuracy = (tp+tn)/all
print(accuracy)

0.5226130653266332


###ANSWER: 0.5226130653266332


***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [ ]:
# YOUR CODE HERE
Y_train = pd.get_dummies(df_train['gender']).iloc[:, 0].to_numpy()
Y_test = pd.get_dummies(df_test['gender']).iloc[:, 0].to_numpy()
X_train = df_train['height'].to_numpy().reshape(-1, 1)
X_test = df_test['height'].to_numpy().reshape(-1, 1)


In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (10,), activation= 'logistic', solver='lbfgs', verbose=1, random_state=42)

clf.fit(X_train, Y_train)

y_pred_train = clf.predict(X_train)
q2a_accuracy = accuracy_score(Y_train, y_pred_train)
print(q2a_accuracy)

0.6440251572327044


###ANSWER: 0.6440251572327044


**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [ ]:
# YOUR CODE HERE
X_test = df_test['height'].to_numpy().reshape(-1, 1)
Y_test = pd.get_dummies(df_test['gender']).iloc[:, 0].to_numpy()
y_pred_test = clf.predict(X_test)

q2b_accuracy = accuracy_score(Y_test, y_pred_test)

print(q2b_accuracy)

0.6206030150753769


###ANSWER: 0.6206030150753769


**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature

In [ ]:
# YOUR CODE HERE
scaler_test = StandardScaler()
test_norm = scaler_test.fit_transform(X_test)
scaler_train = StandardScaler()
train_norm = scaler_train.fit_transform(X_train)
clf_ = MLPClassifier(hidden_layer_sizes= 10, activation= 'logistic', solver='lbfgs', random_state=42)
clf_.fit(train_norm, Y_train)

y_norm_train = clf_.predict(train_norm)
q2c_accuracy_train = accuracy_score(Y_train, y_norm_train)
print(q2c_accuracy_train)
y_norm_test = clf_.predict(test_norm)
q2c_accuracy_test = accuracy_score(Y_test, y_norm_test)
print(q2c_accuracy_test)

0.8465408805031447
0.8542713567839196


###ANSWER: 

***

### Question 3###
The rest of features in this dataset barring a few are categorical. No ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [ ]:
# YOUR CODE HERE
df_train['year'] = df_train['year'].replace('first"', 'first')
df_train['year'].value_counts()


second    527
third     522
fourth    251
first     184
other     106
Name: year, dtype: int64

In [ ]:
df_test['year'] = df_test['year'].replace('first"', 'first')
df_test['year'].value_counts()

second    142
third     124
fourth     58
first      46
other      28
Name: year, dtype: int64

In [ ]:
dummies_train = pd.get_dummies(df_train[['year', 'eyecolor', 'exercise']])
dummies_test = pd.get_dummies(df_test[['year', 'eyecolor', 'exercise']])

q3_clf = MLPClassifier(hidden_layer_sizes=(10), activation= 'logistic', solver='lbfgs', random_state=42, max_iter=50)

q3_clf.fit(dummies_train.to_numpy(), Y_train)

q3_Y_pred = q3_clf.predict(dummies_test.to_numpy())

q3_accuracy = accuracy_score(Y_test, q3_Y_pred)
print(q3_accuracy)

0.6030150753768844


###ANSWER: 0.6030150753768844

***
### Question 4###
Using a NN, report the accuracy on  the test set of a model that trained only on the height and the eye color features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original height values (no pre-processing) and eye color as a one-hot?

In [ ]:
# YOUR CODE HERE
q4a_X_train = pd.get_dummies(df_train[['height', 'eyecolor']], prefix=['eyecolor'])
q4a_X_test = pd.get_dummies(df_test[['height', 'eyecolor']], prefix=['eyecolor'])

q4a_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q4a_clf.fit(q4a_X_train.to_numpy(), Y_train)
q4a_Y_pred = q4a_clf.predict(q4a_X_test.to_numpy())

q4a_accuracy = accuracy_score(Y_test, q4a_Y_pred)
print(q4a_accuracy)

0.7412060301507538


###ANSWER: 0.7412060301507538

**Question 4.b**  
What is the accuracy on the test set using the log of height values (applied to both training and testing sets) and eye color as a one-hot?

In [ ]:
# YOUR CODE HERE
q4b_X_train = pd.get_dummies(df_train[['height', 'eyecolor']], prefix=['eyecolor'])
q4b_X_test = pd.get_dummies(df_test[['height', 'eyecolor']], prefix=['eyecolor'])

q4b_X_train['height'] = np.log(q4b_X_train['height'])
q4b_X_test['height'] = np.log(q4b_X_test['height'])
q4b_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q4b_clf.fit(q4b_X_train.to_numpy(), Y_train)
q4b_Y_pred = q4b_clf.predict(q4b_X_test.to_numpy())

q4b_accuracy = accuracy_score(Y_test, q4b_Y_pred)
print(q4b_accuracy)

0.8492462311557789


###ANSWER: 0.8492462311557789

**Question 4.c**  
What is the accuracy on the test set using the Z-score of height values and eye color as a one-hot? 

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [ ]:
# YOUR CODE HERE
q4c_X_train = pd.get_dummies(df_train[['height', 'eyecolor']], prefix=['eyecolor'])
q4c_X_test = pd.get_dummies(df_test[['height', 'eyecolor']], prefix=['eyecolor'])

q4c_train = q4c_X_train['height'].to_numpy()
q4c_test = q4c_X_test['height'].to_numpy()
q4c_train_z = (q4c_train - np.mean(q4c_train))/np.std(q4c_train)
q4c_test_z = (q4c_test - np.mean(q4c_test))/np.std(q4c_test)
q4c_X_train['height'] = q4c_train_z
q4c_X_test['height'] = q4c_test_z

q4c_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q4c_clf.fit(q4c_X_train.to_numpy(), Y_train)
q4c_Y_pred = q4c_clf.predict(q4c_X_test.to_numpy())

q4c_accuracy = accuracy_score(Y_test, q4c_Y_pred)
print(q4c_accuracy)

0.8693467336683417


###ANSWER: 0.8693467336683417

***
### Question 5 ###
Repeat question 4 for exercise hours + eye color

In [ ]:
# Original Values 
q5a_X_train = pd.get_dummies(df_train[['exercisehours', 'eyecolor']], prefix=['eyecolor'])
q5a_X_test = pd.get_dummies(df_test[['exercisehours', 'eyecolor']], prefix=['eyecolor'])
q5a_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q5a_clf.fit(q5a_X_train.to_numpy(), Y_train)
q5a_Y_pred = q5a_clf.predict(q5a_X_test.to_numpy())

q5a_accuracy = accuracy_score(Y_test, q5a_Y_pred)
q5a_accuracy

0.5628140703517588

In [ ]:
# Log of exercise hours 
q5b_X_train = pd.get_dummies(df_train[['exercisehours', 'eyecolor']], prefix=['eyecolor'])
q5b_X_test = pd.get_dummies(df_test[['exercisehours', 'eyecolor']], prefix=['eyecolor'])

q5b_X_train['exercisehours'] = np.log(q5b_X_train['exercisehours']+0.1)
q5b_X_test['exercisehours'] = np.log(q5b_X_test['exercisehours']+0.1)

q5b_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q5b_clf.fit(q5b_X_train.to_numpy(), Y_train)
q5b_Y_pred = q5b_clf.predict(q5b_X_test.to_numpy())

q5b_accuracy = accuracy_score(Y_test, q5b_Y_pred)
print(q5b_accuracy)

0.5703517587939698


In [ ]:
# Z-score
q5c_X_train = pd.get_dummies(df_train[['exercisehours', 'eyecolor']], prefix=['eyecolor'])
q5c_X_test = pd.get_dummies(df_test[['exercisehours', 'eyecolor']], prefix=['eyecolor'])

q5c_train = q5c_X_train['exercisehours'].to_numpy()
q5c_test = q5c_X_test['exercisehours'].to_numpy()
q5c_train_z = (q5c_train - np.mean(q5c_train))/np.std(q5c_train)
q5c_test_z = (q5c_test - np.mean(q5c_test))/np.std(q5c_test)
q5c_X_train['exercisehours'] = q5c_train_z
q5c_X_test['exercisehours'] = q5c_test_z

q5c_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q5c_clf.fit(q5c_X_train.to_numpy(), Y_train)
q5c_Y_pred = q5c_clf.predict(q5c_X_test.to_numpy())

q5c_accuracy = accuracy_score(Y_test, q5c_Y_pred)
print(q5c_accuracy)


0.5703517587939698


bold text###ANSWER: For original exercise hours, the accuracy on the test set is 0.5628140703517588. For the log of exercise hours, the accuracy is 0.5703517587939698. The Z-score accuracy is 0.5703517587939698.

***
### Question 6###
Combine the features from question 3, 4, and 5 (year, eyecolor, exercise, height, exercise hours). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [ ]:
# YOUR CODE HERE
q6_X_train = pd.concat([dummies_train, q4c_X_train['height'], q5c_X_train['exercisehours']], axis = 1)
q6_X_test = pd.concat([dummies_test, q4c_X_test['height'], q5c_X_test['exercisehours']], axis = 1)

q6_clf = MLPClassifier(hidden_layer_sizes=(10), activation='logistic', solver='lbfgs', random_state=42, max_iter=50)
q6_clf.fit(q6_X_train.to_numpy(), Y_train)
q6_Y_pred = q6_clf.predict(q6_X_test.to_numpy())

q6_accuracy = accuracy_score(Y_test, q6_Y_pred)
print(q6_accuracy)

0.8668341708542714


###ANSWER: 

***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [45]:
# YOUR CODE HERE
q7_X_train = pd.concat([dummies_train, q4c_X_train['height'], q5c_X_train['exercisehours']], axis = 1)
q7_X_test = pd.concat([dummies_test, q4c_X_test['height'], q5c_X_test['exercisehours']], axis = 1)

hidden_layer = [(20, 20, 20), (30, 30, 30)]
max_iter = [900, 1000, 1100, 1200, 1300, 1400]
alpha = 10.0 ** -np.arange(8, 12)
activation = ['logistic']
random = [33, 42] + list(range(20, 30))
scores = []
param = []
for x in activation:
  for y in hidden_layer:
    for z in max_iter:
        for j in alpha:
            q7_mlp = MLPClassifier(hidden_layer_sizes=y, activation=x, solver='lbfgs', random_state=33, max_iter= z, alpha=j)
            q7_mlp.fit(q7_X_train.to_numpy(), Y_train)
            q7_Y_pred = q7_mlp.predict(q7_X_test.to_numpy())
            q7_accuracy = accuracy_score(Y_test, q7_Y_pred)
            scores = scores + [[q7_accuracy]]
            param = param + [[x, y, z, j]]
print(scores)
print(param)
# parameter_space = {
#     'hidden_layer_sizes': [(x) for x in range(20, 60)],
#     'activation': ['identity', 'logistic'],
#     'solver': ['lbfgs'],
#     'max_iter': [100, 300, 200, 400],
#     'random_state': [0,1,2,3,4,5,6,7,8,9,42],
#     'alpha': 10.0 ** -np.arange(1, 10)
# }
# q7_clf = GridSearchCV(q7_mlp, param_grid=parameter_space, refit = False)


[[0.907035175879397], [0.9170854271356784], [0.9020100502512562], [0.9221105527638191], [0.907035175879397], [0.9095477386934674], [0.8944723618090452], [0.9170854271356784], [0.907035175879397], [0.9120603015075377], [0.9020100502512562], [0.9246231155778895], [0.9170854271356784], [0.9170854271356784], [0.907035175879397], [0.9221105527638191], [0.914572864321608], [0.914572864321608], [0.9120603015075377], [0.9170854271356784], [0.8969849246231156], [0.914572864321608], [0.9120603015075377], [0.9195979899497487], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959799], [0.7135678391959

In [48]:
np.concatenate([param, scores], axis=1)

array([['logistic', (20, 20, 20), 900, 1e-11, 0.907035175879397],
       ['logistic', (20, 20, 20), 900, 1e-12, 0.9170854271356784],
       ['logistic', (20, 20, 20), 900, 1e-13, 0.9020100502512562],
       ['logistic', (20, 20, 20), 900, 1e-14, 0.9221105527638191],
       ['logistic', (20, 20, 20), 1000, 1e-11, 0.907035175879397],
       ['logistic', (20, 20, 20), 1000, 1e-12, 0.9095477386934674],
       ['logistic', (20, 20, 20), 1000, 1e-13, 0.8944723618090452],
       ['logistic', (20, 20, 20), 1000, 1e-14, 0.9170854271356784],
       ['logistic', (20, 20, 20), 1100, 1e-11, 0.907035175879397],
       ['logistic', (20, 20, 20), 1100, 1e-12, 0.9120603015075377],
       ['logistic', (20, 20, 20), 1100, 1e-13, 0.9020100502512562],
       ['logistic', (20, 20, 20), 1100, 1e-14, 0.9246231155778895],
       ['logistic', (20, 20, 20), 1200, 1e-11, 0.9170854271356784],
       ['logistic', (20, 20, 20), 1200, 1e-12, 0.9170854271356784],
       ['logistic', (20, 20, 20), 1200, 1e-13, 0.907035

In [50]:
max(scores)

[0.9396984924623115]

In [51]:
hidden_layer = [(30, 30, 30), (40, 40, 40)]
max_iter = [1100, 1200, 1300, 1400]
alpha = 10.0 ** -np.arange(14, 17)
activation = ['logistic']
random = [33, 34, 35, 36, 42]
scores = []
param = []
for x in random:
  for y in hidden_layer:
    for z in max_iter:
        for j in alpha:
            q7_mlp = MLPClassifier(hidden_layer_sizes=y, activation='logistic', solver='lbfgs', random_state=x, max_iter= z, alpha=j)
            q7_mlp.fit(q7_X_train.to_numpy(), Y_train)
            q7_Y_pred = q7_mlp.predict(q7_X_test.to_numpy())
            q7_accuracy = accuracy_score(Y_test, q7_Y_pred)
            scores = scores + [[q7_accuracy]]
            param = param + [[x, y, z, j]]
print(scores)
print(param)

[[0.9120603015075377], [0.9095477386934674], [0.9020100502512562], [0.914572864321608], [0.9170854271356784], [0.8969849246231156], [0.914572864321608], [0.9221105527638191], [0.8944723618090452], [0.9045226130653267], [0.914572864321608], [0.8994974874371859], [0.914572864321608], [0.9271356783919598], [0.9221105527638191], [0.907035175879397], [0.9271356783919598], [0.9120603015075377], [0.9095477386934674], [0.9346733668341709], [0.914572864321608], [0.9120603015075377], [0.9271356783919598], [0.9095477386934674], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.5778894472361809], [0.9271356783919598], [0.9246231155778895], [0.9271356783919598], [0.9246231155778895], [0.9271356783919598], [0.9296482412060302], [0.9221105527638191], [0.9346733668341709], [0.9246231155778895], [0.919597989949

In [52]:
np.concatenate([param, scores], axis=1)

array([[33, (30, 30, 30), 1100, 1e-14, 0.9120603015075377],
       [33, (30, 30, 30), 1100, 1e-15, 0.9095477386934674],
       [33, (30, 30, 30), 1100, 1e-16, 0.9020100502512562],
       [33, (30, 30, 30), 1200, 1e-14, 0.914572864321608],
       [33, (30, 30, 30), 1200, 1e-15, 0.9170854271356784],
       [33, (30, 30, 30), 1200, 1e-16, 0.8969849246231156],
       [33, (30, 30, 30), 1300, 1e-14, 0.914572864321608],
       [33, (30, 30, 30), 1300, 1e-15, 0.9221105527638191],
       [33, (30, 30, 30), 1300, 1e-16, 0.8944723618090452],
       [33, (30, 30, 30), 1400, 1e-14, 0.9045226130653267],
       [33, (30, 30, 30), 1400, 1e-15, 0.914572864321608],
       [33, (30, 30, 30), 1400, 1e-16, 0.8994974874371859],
       [33, (40, 40, 40), 1100, 1e-14, 0.914572864321608],
       [33, (40, 40, 40), 1100, 1e-15, 0.9271356783919598],
       [33, (40, 40, 40), 1100, 1e-16, 0.9221105527638191],
       [33, (40, 40, 40), 1200, 1e-14, 0.907035175879397],
       [33, (40, 40, 40), 1200, 1e-15, 0.9271

In [53]:
max(scores)

[0.9346733668341709]

In [58]:
hidden_layer = (30, 30, 30)
max_iter = 1200
alpha = 10.0 ** -11
activation = ['logistic']
random = range(20, 30)
scores = []
param = []
for x in random:
            q7_mlp = MLPClassifier(hidden_layer_sizes=hidden_layer, activation='logistic', solver='lbfgs', random_state=x, max_iter= 1200, alpha=10.0 ** -11)
            q7_mlp.fit(q7_X_train.to_numpy(), Y_train)
            q7_Y_pred = q7_mlp.predict(q7_X_test.to_numpy())
            q7_accuracy = accuracy_score(Y_test, q7_Y_pred)
            scores = scores + [[q7_accuracy]]
            param = param + [x]
print(scores)
print(param)

[[0.914572864321608], [0.9321608040201005], [0.8994974874371859], [0.9195979899497487], [0.9296482412060302], [0.9321608040201005], [0.5226130653266332], [0.914572864321608], [0.9170854271356784], [0.9296482412060302]]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


###ANSWER: I was able to improve the accuracy by more than 5% (from 86% to 94%). To improve accuracy through tuning, I adopted the approach similar to Gridsearch, where I define a parameter space and used for loop to check the model accuracy for each set of parameters.Through increasing max_iter and hidden layer sizes and decreasing alpha, I was able to increase accuracy by 8%. 

In [59]:
q7_mlp = MLPClassifier(hidden_layer_sizes=(30, 30, 30), activation='logistic', solver='lbfgs', random_state=33, max_iter= 1200, alpha=10.0 ** -11)
q7_mlp.fit(q7_X_train.to_numpy(), Y_train)
q7_Y_pred = q7_mlp.predict(q7_X_test.to_numpy())
q7_accuracy = accuracy_score(Y_test, q7_Y_pred)
print(q7_accuracy)

0.9396984924623115
